In [1]:
!pip install geopy
!pip install folium
!pip install geocoder
print("Installed!")

     |████████████████████████████████| 112kB 6.9MB/s eta 0:00:01
     |████████████████████████████████| 102kB 11.4MB/s ta 0:00:01
     |████████████████████████████████| 829kB 5.2MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d2/3d/fa76db83bf75c4f8d338c2fd15c8d33fdd7ad23a9b5e57eb6c5de26b430e/click-7.1.2-py2.py3-none-any.whl
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
Installed!


In [2]:
import pandas as pd
import numpy as np
import geocoder as GC
print("Successfully Imported!")

Successfully Imported!


In [3]:
import folium
import requests 
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors


from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print("Libraries Successfully Imported!")

Libraries Successfully Imported!


### After successful installation and importation of the necessary libraries, we will proceed to reading our data into a pandas dataframe

In [4]:
Toronto = pd.read_csv('toronto_part2.csv')
Toronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.81153,-79.19552
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.78564,-79.15871
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.76575,-79.17520
3,M1G,Scarborough,Woburn,43.76820,-79.21761
4,M1H,Scarborough,Cedarbrae,43.76969,-79.23944


In [5]:
NewYork = pd.read_csv('NewYork.csv')
NewYork.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


### Use geopy library to get the latitude and longitude values of Toronto and New York City. Then create a map of NewYork and Toronto with neighborhoods superimposed on them  

In [7]:
address = 'Toronto, Ontario Canada'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude1 = location.latitude
longitude1 = location.longitude
print('The geograpical coordinate of Toronto Canada are {}, {}.'.format(latitude1, longitude1))

The geograpical coordinate of Toronto Canada are 43.6534817, -79.3839347.


In [8]:
map_toronto = folium.Map(location=[latitude1, longitude1], zoom_start=11)
folium.features.CircleMarker(
    [latitude1, longitude1],
    radius=10,
    color='red',
    popup='Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_toronto)
for lat, lng, borough, neighborhood in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Borough'], Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)
map_toronto

In [9]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [10]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='New York City',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_newyork)

# add markers to map
for lat, lng, borough, neighborhoods in zip(NewYork['Latitude'], NewYork['Longitude'], NewYork['Borough'], NewYork['Neighborhood']):
    label = '{}, {}'.format(neighborhoods, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
map_newyork

### Select all Etobicoke Boroughs in the Toronto dataframe and all Brooklyn Boroughs in NewYork dataframe and read them into a separate dataframe 

In [11]:
Eb = Toronto[Toronto['Borough'].str.contains("Etobicoke")].reset_index(drop=True)
print(Eb.shape)
Eb.head()

(12, 5)


,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.60987,-79.49693
1,M8W,Etobicoke,"Alderwood, Long Branch",43.60108,-79.53891
2,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65369,-79.51112
3,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.63288,-79.48955
4,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.62463,-79.52835


In [12]:
Bk = NewYork[NewYork['Borough'].str.contains("Brooklyn")].reset_index(drop=True)
print(Bk.shape)
Bk.head()

(70, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


### Create a map of Etobicoke Borough for Toronto and Brooklyn for NewYork

In [13]:
map_toronto = folium.Map(location=[latitude1, longitude1], zoom_start=11)
folium.features.CircleMarker(
    [latitude1, longitude1],
    radius=10,
    color='red',
    popup='Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_toronto)

for lat, lng, neighborhood, borough in zip(Eb['Latitude'], Eb['Longitude'], Eb['Neighborhood'], Eb['Borough']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7,parse_html=False).add_to(map_toronto)  
map_toronto

In [15]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=11)
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='New York City',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(map_newyork)

for lat, lng, neighborhood, borough in zip(Bk['Latitude'], Bk['Longitude'], Bk['Neighborhood'], Bk['Borough']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7,parse_html=False).add_to(map_newyork)  
map_newyork

### Using the FOURSQUARE API to get some location data for our dataframes of Etobicoke and NewYork

In [16]:
CLIENT_ID = '5QS5YKJ3LB5BVYHZW5KAI4434QW4KFGKCEAU1AFZKIY5WMTH' # Put Your Client Id
CLIENT_SECRET = '5XAKYCTJDTS5CI0TTOC5CHOHQETAH3CFOENC1HOZW1XACLM0' # Put You Client Secret 
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: N/A')
print('CLIENT_SECRET: N/A')

Your credentails:
CLIENT_ID: N/A
CLIENT_SECRET: N/A


### Select the first neighborhood in the Etobicoke and Brooklyn dataframes respectively

In [17]:
Eb.loc[0, 'Neighborhood']

'New Toronto, Mimico South, Humber Bay Shores'

In [18]:
Bk.loc[0, 'Neighborhood']

'Bay Ridge'

In [19]:
neighborhood_latitude1 = Eb.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude1 = Eb.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name1 = Eb.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name1, 
                                                               neighborhood_latitude1, 
                                                               neighborhood_longitude1))

Latitude and longitude values of New Toronto, Mimico South, Humber Bay Shores are 43.609870000000065, -79.49692999999995.


In [20]:
neighborhood_latitude = Bk.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Bk.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Bk.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bay Ridge are 40.625801065010656, -74.03062069353813.


### Create a url to make the API request for 50 venues within 700 meters radius for the first neighborhood of Etobicoke and 10 venues within 700 meters for the first neighborhood of Brooklyn respectively

In [21]:
LIMIT = 50 
radius = 700
url1 = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude1, 
    neighborhood_longitude1, 
    radius, 
    LIMIT)
url1

'https://api.foursquare.com/v2/venues/explore?&client_id=5QS5YKJ3LB5BVYHZW5KAI4434QW4KFGKCEAU1AFZKIY5WMTH&client_secret=5XAKYCTJDTS5CI0TTOC5CHOHQETAH3CFOENC1HOZW1XACLM0&v=20180604&ll=43.609870000000065,-79.49692999999995&radius=700&limit=50'

In [30]:
LIMIT = 10 
radius = 700
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=5QS5YKJ3LB5BVYHZW5KAI4434QW4KFGKCEAU1AFZKIY5WMTH&client_secret=5XAKYCTJDTS5CI0TTOC5CHOHQETAH3CFOENC1HOZW1XACLM0&v=20180604&ll=40.625801065010656,-74.03062069353813&radius=700&limit=10'

### Get the results as a json file

In [23]:
results1 = requests.get(url1).json()

In [31]:
results = requests.get(url).json()

In [25]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Process and convert the dataframe into a pandas dataframe

In [26]:
venues1 = results1['response']['groups'][0]['items']
    
nearby_venues1 = json_normalize(venues1) # flatten JSON

# filter columns
filtered_columns1 = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues1 =nearby_venues1.loc[:, filtered_columns1]

# filter the category for each row
nearby_venues1['venue.categories'] = nearby_venues1.apply(get_category_type, axis=1)

# clean columns
nearby_venues1.columns = [col.split(".")[-1] for col in nearby_venues1.columns]

nearby_venues1.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Mario & Selina's No Frills,Grocery Store,43.612186,-79.497828
1,Mimico Arena,Skating Rink,43.612739,-79.498682
2,TD Canada Trust,Bank,43.613182,-79.489958
3,7-Eleven,Convenience Store,43.613076,-79.490047
4,Hasty Market,Convenience Store,43.611390,-79.496398


In [32]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Pilo Arts Day Spa and Salon,Spa,40.624748,-74.030591
1,Bagel Boy,Bagel Shop,40.627896,-74.029335
2,Pegasus Cafe,Breakfast Spot,40.623168,-74.031186
3,Ho' Brah Taco Joint,Taco Place,40.622960,-74.031371
4,Leo's Casa Calamari,Pizza Place,40.624200,-74.030931


In [33]:
print('{} venues were returned by Foursquare API for the first neighborhood in Etobicoke.'.format(nearby_venues1.shape[0]))

6 venues were returned by Foursquare API for the first neighborhood in Etobicoke.


In [34]:
print('{} venues were returned by Foursquare API for the first neighborhood in Brooklyn.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare API for the first neighborhood in Brooklyn.


### Explore all other neighborhoods in the dataframes (Etobicoke and Brooklyn) for nearby venues as we did for the first neighborhood of both Etobicoke and Brooklyn

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([( 
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
Etobicoke_venues = getNearbyVenues(names=Eb['Neighborhood'],
                                   latitudes=Eb['Latitude'],
                                   longitudes=Eb['Longitude']
                                  )

New Toronto, Mimico South, Humber Bay Shores
Alderwood, Long Branch
The Kingsway, Montgomery Road, Old Mill North
Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East
Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West
Islington Avenue, Humber Valley Village
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Westmount
Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens
South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens
Northwest, West Humber - Clairville


In [37]:
Brooklyn_venues = getNearbyVenues(names=Bk['Neighborhood'],
                                   latitudes=Bk['Latitude'],
                                   longitudes=Bk['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [38]:
print(Etobicoke_venues.shape)
Etobicoke_venues.head()

(105, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"New Toronto, Mimico South, Humber Bay Shores",43.60987,-79.49693,Mario & Selina's No Frills,43.612186,-79.497828,Grocery Store
1,"New Toronto, Mimico South, Humber Bay Shores",43.60987,-79.49693,Mimico Arena,43.612739,-79.498682,Skating Rink
2,"New Toronto, Mimico South, Humber Bay Shores",43.60987,-79.49693,TD Canada Trust,43.613182,-79.489958,Bank
3,"New Toronto, Mimico South, Humber Bay Shores",43.60987,-79.49693,7-Eleven,43.613076,-79.490047,Convenience Store
4,"New Toronto, Mimico South, Humber Bay Shores",43.60987,-79.49693,Hasty Market,43.611390,-79.496398,Convenience Store


In [39]:
print(Brooklyn_venues.shape)
Brooklyn_venues.head()

(686, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
3,Bay Ridge,40.625801,-74.030621,Ho' Brah Taco Joint,40.622960,-74.031371,Taco Place
4,Bay Ridge,40.625801,-74.030621,Leo's Casa Calamari,40.624200,-74.030931,Pizza Place


In [40]:
Etobicoke_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Alderwood, Long Branch",9,9,9,9,9,9
"Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood",10,10,10,10,10,10
"Islington Avenue, Humber Valley Village",10,10,10,10,10,10
"Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens",6,6,6,6,6,6
"Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West",10,10,10,10,10,10
"New Toronto, Mimico South, Humber Bay Shores",6,6,6,6,6,6
"Northwest, West Humber - Clairville",9,9,9,9,9,9
"Old Mill South, King's Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East",10,10,10,10,10,10
"South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens",10,10,10,10,10,10


In [41]:
Brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,10,10,10,10,10,10
Bay Ridge,10,10,10,10,10,10
Bedford Stuyvesant,10,10,10,10,10,10
Bensonhurst,10,10,10,10,10,10
Bergen Beach,8,8,8,8,8,8
Boerum Hill,10,10,10,10,10,10
Borough Park,10,10,10,10,10,10
Brighton Beach,10,10,10,10,10,10
Broadway Junction,10,10,10,10,10,10


In [42]:
print('There are {} uniques categories of venues in Etobicoke.'.format(len(Etobicoke_venues['Venue Category'].unique())))

There are 53 uniques categories of venues in Etobicoke.


In [43]:
print('There are {} uniques categories of venues in Brooklyn.'.format(len(Brooklyn_venues['Venue Category'].unique())))

There are 171 uniques categories of venues in Brooklyn.


### Analyze each neighborhood

In [44]:
Etobicoke_onehot = pd.get_dummies(Etobicoke_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Etobicoke_onehot['Neighborhood'] = Etobicoke_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Etobicoke_onehot.columns[-1]] + list(Etobicoke_onehot.columns[:-1])
Etobicoke_onehot = Etobicoke_onehot[fixed_columns]

Etobicoke_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,Auto Dealership,Automotive Shop,Bakery,Bank,Baseball Field,Beer Store,Breakfast Spot,Burger Joint,Burrito Place,Bus Line,Business Service,Café,Chinese Restaurant,Coffee Shop,College Rec Center,Convenience Store,Dessert Shop,Discount Store,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,French Restaurant,Fried Chicken Joint,Gas Station,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hotel,Italian Restaurant,Liquor Store,Middle Eastern Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Print Shop,Pub,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoothie Shop,Storage Facility,Supermarket,Sushi Restaurant,Thai Restaurant,Toy / Game Store
0,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"New Toronto, Mimico South, Humber Bay Shores",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [45]:
Brooklyn_onehot = pd.get_dummies(Brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Brooklyn_onehot['Neighborhood'] = Brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Brooklyn_onehot.columns[-1]] + list(Brooklyn_onehot.columns[:-1])
Brooklyn_onehot = Brooklyn_onehot[fixed_columns]

Brooklyn_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beer Garden,Big Box Store,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Child Care Service,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Event Service,Event Space,Factory,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,History Museum,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Optical Shop,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pub,Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Plaza,Skate Park,Skating Rink,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Steakhouse,Street Art,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Turkish Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Bay Ridge,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [46]:
Etobicoke_onehot.shape

(105, 54)

In [47]:
Brooklyn_onehot.shape

(686, 171)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [48]:
Etobicoke_grouped = Etobicoke_onehot.groupby('Neighborhood').mean().reset_index()
Etobicoke_grouped

,Neighborhood,American Restaurant,Asian Restaurant,Auto Dealership,Automotive Shop,Bakery,Bank,Baseball Field,Beer Store,Breakfast Spot,Burger Joint,Burrito Place,Bus Line,Business Service,Café,Chinese Restaurant,Coffee Shop,College Rec Center,Convenience Store,Dessert Shop,Discount Store,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,French Restaurant,Fried Chicken Joint,Gas Station,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Hardware Store,Hotel,Italian Restaurant,Liquor Store,Middle Eastern Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Print Shop,Pub,Restaurant,Sandwich Place,Shopping Mall,Skating Rink,Smoothie Shop,Storage Facility,Supermarket,Sushi Restaurant,Thai Restaurant,Toy / Game Store
0,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.111111,0.0,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.000000,0.111111,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.111111,0.111111,0.111111,0.111111,0.000000,0.111111,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.1,0.100000,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.000000,0.0,0.100000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.1,0.000000,0.100000,0.000000,0.000000,0.000000,0.100000,0.1,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
2,"Islington Avenue, Humber Valley Village",0.000000,0.0,0.000000,0.000000,0.1,0.100000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.1,0.0,0.000000,0.0,0.100000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.100000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.100000,0.0,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.1,0.100000,0.0,0.000000,0.0,0.0,0.0,0.0
3,"Kingsview Village, St. Phillips, Martin Grove ...",0.166667,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.166667,0.166667,0.0,0.0,0.166667,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.166667,0.000000,0.000000,0.000000,0.000000,0.166667,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
4,"Mimico NW, The Queensway West, South of Bloor,...",0.000000,0.1,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.1,0.2,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.000000,0.0,0.000000,0.100000,0.1,0.0,0.0,0.0,0.0,0.100000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.1,0.1,0.0
5,"New Toronto, Mimico South, Humber Bay Shores",0.000000,0.0,0.000000,0.000000,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.166667,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.166667,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.166667,0.0,0.000000,0.0,0.0,0.0,0.0
6,"Northwest, West Humber - Clairville",0.000000,0.0,0.111111,0.111111,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.111111,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.000000,0.111111,0.0,0.0,0.0,0.0,0.0,0.111111,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.111111,0.111111,0.0,0.000000,0.0,0.111111,0.0,0.0,0.0,0.0
7,"Old Mill South, King's Mill Park, Sunnylea, Hu...",0.000000,0.0,0.000000,0.000000,0.0,0.100000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.200000,0.0,0.000000,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.1,0.1,0.000000,0.000000,0.0,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.1,0.000000,0.0,0.1,0.0,0.0
8,"South Steeles, Silverstone, Humbergate, Jamest...",0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.1,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.100000,0.0,0.000000,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.000000,0.0,0.200000,0.000000,0.0,0.1,0.0,0.0,0.0,0.000000,0.000000,0.0,0.100000,0.100000,0.000000,0.000000,0.000000,0.100000,0.0,0.00000

In [49]:
Brooklyn_grouped = Brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
Brooklyn_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beach,Beer Garden,Big Box Store,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Child Care Service,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Donut Shop,Event Service,Event Space,Factory,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gay Bar,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Health & Beauty Service,Health Food Store,History Museum,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Optical Shop,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Polish Restaurant,Pool,Pub,Restaurant,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Plaza,Skate Park,Skating Rink,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Steakhouse,Street Art,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tourist Information Center,Toy / Game Store,Track,Trail,Turkish Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint
0,Bath Beach,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.1,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Bay Ridge,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.1,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.000,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Bedford Stuyvesant,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.1,0.000,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0

In [50]:
Etobicoke_grouped.shape

(12, 54)

In [51]:
Brooklyn_grouped.shape

(70, 171)

### Let's print each neighborhood along with the top 5 most common venues for both Etobicoke and Brooklyn

In [52]:
num_top_venues = 5

for hood in Etobicoke_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Etobicoke_grouped[Etobicoke_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alderwood, Long Branch----
               venue  freq
0           Pharmacy  0.11
1     Sandwich Place  0.11
2                Gym  0.11
3  Convenience Store  0.11
4        Coffee Shop  0.11


----Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood----
               venue  freq
0        Pizza Place   0.1
1  Convenience Store   0.1
2          Pet Store   0.1
3      Grocery Store   0.1
4  Electronics Store   0.1


----Islington Avenue, Humber Valley Village----
           venue  freq
0       Pharmacy   0.2
1  Grocery Store   0.1
2         Bakery   0.1
3           Bank   0.1
4           Café   0.1


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                 venue  freq
0  American Restaurant  0.17
1       Sandwich Place  0.17
2             Bus Line  0.17
3     Business Service  0.17
4             Pharmacy  0.17


----Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West----
              venue  fre

In [53]:
num_top_venues = 5

for hood in Brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Brooklyn_grouped[Brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                  venue  freq
0        Ice Cream Shop   0.1
1      Sushi Restaurant   0.1
2             Surf Spot   0.1
3            Hookah Bar   0.1
4  Cantonese Restaurant   0.1


----Bay Ridge----
            venue  freq
0   Grocery Store   0.2
1             Spa   0.2
2      Taco Place   0.1
3       Juice Bar   0.1
4  Breakfast Spot   0.1


----Bedford Stuyvesant----
                 venue  freq
0   Italian Restaurant   0.1
1                  Bar   0.1
2  Fried Chicken Joint   0.1
3            Gift Shop   0.1
4             Boutique   0.1


----Bensonhurst----
                venue  freq
0  Italian Restaurant   0.3
1          Bagel Shop   0.2
2              Bakery   0.2
3         Supermarket   0.1
4      Cosmetics Shop   0.1


----Bergen Beach----
                venue  freq
0     Harbor / Marina  0.25
1                 Gym  0.12
2                Park  0.12
3  Athletics & Sports  0.12
4          Playground  0.12


----Boerum Hill----
                       venue  f

### Let's put that into a pandas dataframe and display the top 10 venues for each neighborhood.

In [54]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [55]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted1 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted1['Neighborhood'] = Etobicoke_grouped['Neighborhood']

for ind in np.arange(Etobicoke_grouped.shape[0]):
    neighborhoods_venues_sorted1.iloc[ind, 1:] = return_most_common_venues(Etobicoke_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted1.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Alderwood, Long Branch",Sandwich Place,Pub,Coffee Shop,Pharmacy,Gas Station,Convenience Store,Pizza Place,Print Shop,Gym,Chinese Restaurant
1,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Pet Store,Sandwich Place,Electronics Store,Grocery Store,Convenience Store,College Rec Center,Pizza Place,Baseball Field,Fish & Chips Shop,Shopping Mall
2,"Islington Avenue, Humber Valley Village",Pharmacy,Bank,Convenience Store,Park,Bakery,Skating Rink,Shopping Mall,Café,Grocery Store,Coffee Shop
3,"Kingsview Village, St. Phillips, Martin Grove ...",American Restaurant,Sandwich Place,Coffee Shop,Business Service,Bus Line,Pharmacy,Bank,Chinese Restaurant,Fish & Chips Shop,Fast Food Restaurant
4,"Mimico NW, The Queensway West, South of Bloor,...",Burrito Place,Middle Eastern Restaurant,Burger Joint,Sushi Restaurant,Asian Restaurant,Thai Restaurant,Fish Market,Gym,Gym / Fitness Center,College Rec Center


In [56]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Brooklyn_grouped['Neighborhood']

for ind in np.arange(Brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Sushi Restaurant,Bakery,Burger Joint,Cantonese Restaurant,Shopping Plaza,Park,Surf Spot,Ice Cream Shop,Hookah Bar,Pizza Place
1,Bay Ridge,Spa,Grocery Store,Middle Eastern Restaurant,Breakfast Spot,Pizza Place,Taco Place,Juice Bar,Bagel Shop,Field,Food Court
2,Bedford Stuyvesant,Bar,Japanese Restaurant,New American Restaurant,Boutique,Gift Shop,Park,Juice Bar,Fried Chicken Joint,Italian Restaurant,Wine Shop
3,Bensonhurst,Italian Restaurant,Bagel Shop,Bakery,Gourmet Shop,Cosmetics Shop,Supermarket,Wings Joint,Food Truck,Food Court,Food & Drink Shop
4,Bergen Beach,Harbor / Marina,Playground,Athletics & Sports,Comfort Food Restaurant,Gym,Park,Baseball Field,Wings Joint,Fast Food Restaurant,Food Court


### Peter, the international business man can then from this groupings of the ten most common venues in all the neighbourhoods of Etobicoke, Toronto and Brooklyn Newyork pick a more favourable and pleasing venue for his business based on whatever parameters he deems fit. The location data has been carefully analysed and cleaned and insight has been given to him